# 🛡️ Introduzione a **Nemo Guardrails** – Sicurezza per LLM Conversazionali

## 💡 Cos'è Nemo Guardrails?

**Nemo Guardrails** è un toolkit **open source** progettato per:

* Aggiungere **guardrail programmabili** (binari di sicurezza) alle applicazioni **LLM-based**.
* Proteggere da comportamenti indesiderati, **jailbreak**, **iniezioni di prompt**, output inappropriati, ecc.

### 🎯 Obiettivi principali:

| Obiettivo                   | Descrizione                                                       |
| --------------------------- | ----------------------------------------------------------------- |
| ✅ Affidabilità              | Controllo sull’output dell’LLM in contesti critici                |
| 🛡️ Sicurezza               | Protezione contro attacchi (es. SQL injection, jailbreak)         |
| 🧠 Guidare le conversazioni | Definire **flussi conversazionali** chiari e tracciabili          |
| 🔄 Standardizzazione        | Applicare best practices, SOP, autenticazioni, percorsi specifici |

---

## 🧰 Componenti principali di Nemo Guardrails

### 🧱 1. `Colang` – Language per il design conversazionale

* Linguaggio dichiarativo per **modellare conversazioni**.
* Concetti chiave: `messages` e `flows`.

### 📄 2. YAML – Configurazione dei modelli

* Imposta il modello da usare (es. GPT-3.5-Turbo)
* Specifica temperature, max\_tokens, ecc.

---

## 🧪 Esempio: Flusso di saluto

### 🔹 Messaggio dell’utente (`colang`)

```colang
define user express greeting
  "hello"
  "hi"

define bot express greeting
  "Hello. How can I help you today?"
```

### 🔹 Flusso conversazionale (`colang`)

```colang
flow hello:
  user express greeting
  bot express greeting
```

### 🔹 Configurazione YAML

```yaml
models:
  - type: openai
    engine: gpt-3.5-turbo
    api_key: ${OPENAI_API_KEY}
```

---

## 🧑‍💻 Implementazione in Jupyter Notebook

### 🔧 Setup iniziale

Dobbiamo modificare il ciclo asyncIO per far funzionare NeMo guardrails in un ambiente di Jupyter Notebook.

```python
import nest_asyncio
nest_asyncio.apply()
```

## 🔄 Variabili e logica condizionale (if / else)

### 🔹 Nuovo colang con variabili

```colang
define user express greeting
  "hello"
  "hi"

define bot express normal greeting
  "Hello. How can I help you today?"

define bot express personal greeting
  "Hello $username. It's nice to see you again."
```

### 🔹 Flusso condizionale

```colang
flow hello:
  user express greeting
  if $username:
    bot express personal greeting
  else:
    bot express normal greeting
```

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import nest_asyncio

nest_asyncio.apply()

**Colang** è un linguaggio di modellazione per applicazioni conversazionali

Utilizziamo Colang per progettare il modo in cui deve avvenire la conversazione tra un utente e un bot.

In Colang i due concetti fondamentali sono i **messaggi** e i **flussi**

Una conversazione viene modellata come uno scambio di un messaggio dell'utente e di un messaggio del bot.

Per farlo funzionare dobbiamo anche definire il `yaml_content` che definisce il modello che useremo.

In [3]:
# potrebbero essre anche file YAML

colang_content = """
define user express greeting
    "hello"
    "hi"
    
define bot express greeting
    "Hello there!! Can I help you today?"
    
define flow hello
    user express greeting
    bot express greeting
"""

yaml_content = """
models:
- type: main
  engine: openai
  model: gpt-3.5-turbo
"""

from nemoguardrails import LLMRails, RailsConfig

config = RailsConfig.from_content(
    yaml_content=yaml_content, colang_content=colang_content
)



In [4]:
rails = LLMRails(config=config)

c:\Users\felip\Desktop\Advanced_RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:17<00:00,  3.55s/it]



### 🔁 Generazione della risposta

```python
response = await rails.generate_async("hello")
print(response)
# Output: Hello. How can I help you today?
```

In [5]:
res = await rails.generate_async(prompt="Hello")

print(res)

Hello there!! Can I help you today?


---

## 🧩 Modalità conversazione (multi-turn)

Invece di passare solo un prompt, possiamo anche passare una conversazione completa simile a come facciamo con le OpenAI API.

```python
await rails.generate_async([
    {"role": "user", "content": "hello"}
])
```

### ⏱️ Output:

```json
{"role": "assistant", "content": "Hello. How can I help you today?"}
```

In [6]:
messages = [{"role": "user", "content": "Hey there!"}]

res = await rails.generate_async(messages=messages)

print(res)

{'role': 'assistant', 'content': 'Hello there!! Can I help you today?'}


---

## 🎯 Possiamo rendere il Colang un po' più complesso

Possiamo usare anche variabili per rendere il tutto più dinamico.

Usaimo la variabile $username per indicare che il saluto personale deve essere rivolto ad un user in particolare.

In [12]:
colang_content = """
define user express greeting
    "hello"
    "hi"

define bot express greeting
    "Hello there!! Can I help you today?"

define bot personal greeting
    "Hello $username, nice to see you again!"

define flow hello
    user express greeting
    if $username
        bot personal greeting
    else
        bot express greeting
"""

In [13]:
config = RailsConfig.from_content(
    yaml_content=yaml_content, colang_content=colang_content
)

rails = LLMRails(config=config)

In [14]:
messages = [{"role": "user", "content": "Hey there!"}]

res = await rails.generate_async(messages=messages)

print(res)

{'role': 'assistant', 'content': 'Hello there!! Can I help you today?'}


In [ ]:
messages = [
    {"role": "context", "content": {"username": "Markus"}},
    {"role": "user", "content": "Hey there!"}
]

res = await rails.generate_async(messages=messages)

print(res)

{'role': 'assistant', 'content': 'Hello Markus, nice to see you again!'}


: 

---

## 🧱 Riepilogo dei concetti chiave

| Concetto         | Descrizione                                                 |
| ---------------- | ----------------------------------------------------------- |
| `colang`         | Linguaggio per definire esempi e flussi conversazionali     |
| `flow`           | Logica condizionale delle conversazioni                     |
| `YAML`           | Configura i modelli da usare e i parametri                  |
| `RailsConfig`    | Oggetto che unisce YAML + Colang                            |
| `LLMRails`       | Motore per eseguire i guardrail                             |
| `generate_async` | Metodo asincrono per generare output con i vincoli definiti |

---

## ✅ Conclusione

Hai imparato a:

* Usare **Nemo Guardrails** per controllare l’output del tuo chatbot.
* Modellare una conversazione con `colang`.
* Eseguire logiche condizionali e usare variabili di contesto.
* Creare un sistema LLM più **robusto, sicuro e prevedibile**.

> 🔜 Nella prossima lezione imparerai a **integrare Nemo Guardrails con LangChain** per un controllo conversazionale ancora più potente.